In [ ]:
#!/usr/bin/env python
# coding: utf-8

import urllib.request
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd
import re
import os
import pickle

In [ ]:
# Step 1: Extract Locus IDs from the .tex file
def extract_locus_ids(tex_file_path):
    locus_ids = []
    with open(tex_file_path, 'r') as file:
        content = file.read()
        locus_ids = re.findall(r'PA14_\d{5}', content)
    return locus_ids

In [ ]:
# just checking

tex_file_path = '/content/ES_master_list.txt'
locus_ids = extract_locus_ids(tex_file_path)
print(locus_ids)
print(len(locus_ids))

['PA14_00010', 'PA14_00020', 'PA14_00030', 'PA14_00050', 'PA14_00060', 'PA14_00070', 'PA14_00090', 'PA14_00100', 'PA14_00120', 'PA14_00190', 'PA14_00200', 'PA14_00240', 'PA14_00280', 'PA14_00290', 'PA14_00440', 'PA14_00450', 'PA14_00800', 'PA14_01130', 'PA14_01510', 'PA14_02620', 'PA14_02690', 'PA14_02900', 'PA14_02970', 'PA14_03163', 'PA14_04110', 'PA14_04140', 'PA14_04310', 'PA14_04380', 'PA14_04460', 'PA14_04480', 'PA14_04580', 'PA14_04630', 'PA14_04750', 'PA14_04760', 'PA14_04900', 'PA14_04930', 'PA14_04980', 'PA14_05070', 'PA14_05080', 'PA14_05150', 'PA14_05250', 'PA14_05260', 'PA14_05280', 'PA14_05460', 'PA14_05550', 'PA14_05590', 'PA14_05620', 'PA14_05775', 'PA14_06500', 'PA14_06510', 'PA14_06530', 'PA14_06540', 'PA14_06570', 'PA14_06600', 'PA14_07090', 'PA14_07130', 'PA14_07170', 'PA14_07190', 'PA14_07230', 'PA14_07520', 'PA14_07530', 'PA14_07560', 'PA14_07570', 'PA14_07590', 'PA14_07600', 'PA14_07620', 'PA14_07700', 'PA14_07740', 'PA14_07760', 'PA14_07770', 'PA14_07910', 'PA14

In [ ]:
search_url_template = "https://pseudomonas.com/primarySequenceFeature/list?strain_ids=109&term=Pseudomonas+aeruginosa+UCBPP-PA14+%28Lee+et+al.%2C+2006%29&c1=name&v1={locus_id}&e1=1&assembly=complete"


In [ ]:
# extracting IDs to use when looking for specific page that includes the sequences

def fetch_url(locus_id):
    search_url = search_url_template.format(locus_id=locus_id)
    response = requests.get(search_url)
    if response.status_code == 200:
        return response.text
    else:
        print(f"Failed to retrieve page for Locus ID: {locus_id}. Status code: {response.status_code}")
        return None

def extract_feature_id_from_html(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    feature_ids = set()  # Use a set to avoid duplicates
    # Find all <a> tags with href containing 'feature/show'
    for a_tag in soup.find_all('a', href=True):
        href = a_tag['href']
        if 'feature/show' in href:
            # Extract the ID from the URL
            match = re.search(r'id=(\d+)', href)
            if match:
                feature_ids.add(match.group(1))  # Add to set
    return feature_ids

# List to store locus_id and corresponding feature_id pairs
locus_feature_pairs = []

# Fetch and process data for each locus_id
for locus_id in locus_ids:
    print(f"Processing Locus ID: {locus_id}")
    html_content = fetch_url(locus_id)
    if html_content:
        feature_ids = extract_feature_id_from_html(html_content)
        if feature_ids:
            for feature_id in feature_ids:
                locus_feature_pairs.append((locus_id, feature_id))
                print(f"Extracted Feature ID for Locus ID {locus_id}: {feature_id}")
        else:
            locus_feature_pairs.append((locus_id, None))  # No feature IDs found
            print(f"No feature IDs found for Locus ID: {locus_id}")
    else:
        locus_feature_pairs.append((locus_id, None))  # No content retrieved

# Save the list of pairs to a .pkl file
with open('locus_feature_pairs.pkl', 'wb') as pkl_file:
    pickle.dump(locus_feature_pairs, pkl_file)

print("Data has been saved to locus_feature_pairs.pkl")


Processing Locus ID: PA14_00010
Extracted Feature ID for Locus ID PA14_00010: 1650835
Processing Locus ID: PA14_00020
Extracted Feature ID for Locus ID PA14_00020: 1650837
Processing Locus ID: PA14_00030
Extracted Feature ID for Locus ID PA14_00030: 1650839
Processing Locus ID: PA14_00050
Extracted Feature ID for Locus ID PA14_00050: 1650841
Processing Locus ID: PA14_00060
Extracted Feature ID for Locus ID PA14_00060: 1650843
Processing Locus ID: PA14_00070
Extracted Feature ID for Locus ID PA14_00070: 1650845
Processing Locus ID: PA14_00090
Extracted Feature ID for Locus ID PA14_00090: 1650849
Processing Locus ID: PA14_00100
Extracted Feature ID for Locus ID PA14_00100: 1650851
Processing Locus ID: PA14_00120
Extracted Feature ID for Locus ID PA14_00120: 1650855
Processing Locus ID: PA14_00190
Extracted Feature ID for Locus ID PA14_00190: 1650869
Processing Locus ID: PA14_00200
Extracted Feature ID for Locus ID PA14_00200: 1650871
Processing Locus ID: PA14_00240
Extracted Feature ID f

In [ ]:
with open('locus_feature_pairs.pkl', 'rb') as pkl_file:
    locus_feature_pairs = pickle.load(pkl_file)

In [ ]:
#just checking to make sure the code is doing what it needs to do
print(len(locus_feature_pairs))
print(locus_feature_pairs)

839
[('PA14_00010', '1650835'), ('PA14_00020', '1650837'), ('PA14_00030', '1650839'), ('PA14_00050', '1650841'), ('PA14_00060', '1650843'), ('PA14_00070', '1650845'), ('PA14_00090', '1650849'), ('PA14_00100', '1650851'), ('PA14_00120', '1650855'), ('PA14_00190', '1650869'), ('PA14_00200', '1650871'), ('PA14_00240', '1650877'), ('PA14_00280', '1650881'), ('PA14_00290', '1650883'), ('PA14_00440', '1650905'), ('PA14_00450', '1650907'), ('PA14_00800', '1650971'), ('PA14_01130', '1651023'), ('PA14_01510', '1651087'), ('PA14_02620', '1651253'), ('PA14_02690', '1651265'), ('PA14_02900', '1651295'), ('PA14_02970', '1651303'), ('PA14_03163', '1651339'), ('PA14_04110', '1651497'), ('PA14_04140', '1651499'), ('PA14_04310', '1651525'), ('PA14_04380', '1651537'), ('PA14_04460', '1651549'), ('PA14_04480', '1651551'), ('PA14_04580', '1651567'), ('PA14_04630', '1651573'), ('PA14_04750', '1651593'), ('PA14_04760', '1651595'), ('PA14_04900', '1651617'), ('PA14_04930', '1651623'), ('PA14_04980', '1651633

In [ ]:
new_locus_feature_pairs= (locus_feature_pairs[:5])

In [ ]:
# Define the base URL
base_url = "https://pseudomonas.com/feature/show/?id="

# Define the function for fetching the HTML content using urllib
def fetch_html(feature_id):
    url = f"{base_url}{feature_id}&view=sequence"
    fp = urllib.request.urlopen(url)
    mybytes = fp.read()
    html_content = mybytes.decode("utf8")
    fp.close()
    return html_content

# Extract the promoter, CDS sequences, and gene name using urllib and split logic
def extract_promoter_cds_gene_sequences(html_content):
    """Extract the promoter, CDS sequences, and gene name using split logic."""

    # Extract DNA sequence upstream of the gene (Promoter)
    try:
        promoter_sequence = html_content.split('DNA Sequence Upstream of \n\t\t\t\t\t Gene\n\t\t\t\t\t \n\n\t\t\t\t\t</td>\n\t\t\t\t\t<td style="padding: 10px;">\n\t\t\t\t\t\t<div style="font-family: Courier;">\n\t\t\t\t\t\t\t')[1].split(' \n\t')[0].split('\n\t')[0]
    except IndexError:
        promoter_sequence = None  # Handle if the split doesn't find the upstream sequence

    # Extract DNA sequence for the gene (CDS)
    try:
        cds_sequence = html_content.split('DNA Sequence for \n\t\t\t\t\t Gene\n\t\t\t\t\t \n\n\t\t\t\t\t</td>\n\t\t\t\t\t<td style="padding: 10px;">\n\t\t\t\t\t\t<div style="font-family: Courier;">\n\t\t\t\t\t\t\t>')[1].split('<BR>\n\t\t\t\t\t\t\t')[1].split(' \n\t')[0].split('\n\t')[0]
    except IndexError:
        cds_sequence = None  # Handle if the split doesn't find the gene sequence

    # Extract the Gene Name from the text in the h3 tag (similar to how "dnaA" was found in the image)
    try:
        gene_name = html_content.split('<h3>')[1].split('(')[1].split(')')[0]
    except IndexError:
        gene_name = None  # Handle if the gene name can't be found

    return promoter_sequence, cds_sequence, gene_name

# Combine promoter, CDS sequences, and extract gene name
def combine_promoter_cds(html_content):
    """
    Extract promoter, CDS sequences, and gene name, then combine promoter and CDS sequences.
    Returns:
        - gene_name: The extracted gene name
        - promoter_sequence: The extracted promoter sequence
        - cds_sequence: The extracted CDS sequence
        - length_cds: The length of the CDS sequence
        - promoter_cds_sequence: The combined promoter + CDS sequence
        - length_promoter_cds: The length of the combined sequence
    """
    # Extract promoter, CDS sequences, and gene name
    promoter_sequence, cds_sequence, gene_name = extract_promoter_cds_gene_sequences(html_content)

    # Combine promoter and CDS sequences
    promoter_cds_sequence = (promoter_sequence if promoter_sequence else '') + (cds_sequence if cds_sequence else '')

    # Calculate the length of the CDS sequence
    length_cds = len(cds_sequence) if cds_sequence else 0

    # Calculate the length of the combined sequence (promoter + CDS)
    length_promoter_cds = len(promoter_cds_sequence)

    return gene_name, promoter_sequence, cds_sequence, length_cds, promoter_cds_sequence, length_promoter_cds

# Create a DataFrame to store the results
data = []
for index, (locus_id, feature_id) in tqdm(enumerate(locus_feature_pairs)):
    print(f"Processing Locus ID: {locus_id}, Feature ID: {feature_id}")
    html_content = fetch_html(feature_id)
    if html_content:
        gene_name, promoter_sequence, cds_sequence, length_cds, promoter_cds_sequence, length_promoter_cds = combine_promoter_cds(html_content)
        data.append({
            'Index': index,
            'Locus ID': locus_id,
            'Gene Name': gene_name,
            'CDS': cds_sequence,
            'Length': length_cds,  # CDS length specifically
            'Promoter': promoter_sequence,
            'Promoter_CDS': promoter_cds_sequence,
            'Length_Promoter_CDS': length_promoter_cds
        })
    else:
        data.append({
            'Index': index,
            'Locus ID': locus_id,
            'Gene Name': None,
            'CDS': None,
            'Length': None,
            'Promoter': None,
            'Promoter_CDS': None,
            'Length_Promoter_CDS': None
        })

# Convert to DataFrame
df = pd.DataFrame(data)

# Save DataFrame to a .pkl file
with open('gene_information.pkl', 'wb') as pkl_file:
    pickle.dump(df, pkl_file)

print("Data has been saved to gene_information.pkl")

0it [00:00, ?it/s]

Processing Locus ID: PA14_00010, Feature ID: 1650835


1it [00:01,  1.48s/it]

Processing Locus ID: PA14_00020, Feature ID: 1650837


2it [00:02,  1.48s/it]

...

Processing Locus ID: PA14_73410, Feature ID: 1662772


838it [22:01,  1.58s/it]

Processing Locus ID: PA14_73420, Feature ID: 1662774


839it [22:03,  1.58s/it]

Data has been saved to gene_information.pkl


In [ ]:
df = pd.read_pickle("/content/gene_information.pkl")
print(df.head())
print(len(df))

   Index    Locus ID  Gene Name  \
0      0  PA14_00010       dnaA   
1      1  PA14_00020       dnaN   
2      2  PA14_00030       recF   
3      3  PA14_00050       gyrB   
4      4  PA14_00060  0, 0, 255   

                                                 CDS  Length  \
0  GTGTCCGTGGAACTTTGGCAGCAGTGCGTGGATCTTCTCCGCGATG...    1560   
1  ATGCATTTCACCATTCAACGCGAAGCCCTGTTGAAACCGCTGCAAC...    1115   
2  ATGTCCCTGACCCGCGTTTCGGTCACCGCGGTGCGCAACCTGCACC...    1121   
3  ATGAGCGAGAACAACACGTACGACTCTTCCAGCATCAAGGTGCTGA...    2445   
4  ATGTCGACAGTGCAGGCCATCAGAACCGTCCTCTTTTACCTGCTGC...     781   

                                            Promoter  \
0  TTTAAAGAGACCGGCGATTCTAGTGAAATCGAACGGGCAGGTCAAT...   
1  GCAGGCGAAGATCGAGCTGCCGCACGATGCGGCCTTCTTCATCGCC...   
2  CTCGTCTGCTCACCGAGCAGGACGGCGAAGTCGGCATCGTCCTGGG...   
3  CCGGTATTCGAGGAAACACTCGCCGAATTGGTTTCACTGGATGACC...   
4  GACTCCGACGATTACATCAAGACCCTCGACGAGTGGATCCCCATCC...   

                                        Promoter_CDS  Length_Promot

In [ ]:
with open('/content/gene_information.pkl', "rb") as f:
    object = pickle.load(f)

df = pd.DataFrame(object)
df.to_csv(r'gene_info.csv')